# оценка точности модели (разбиения документов на предложения)

In [22]:
import os
import pandas as pd
import numpy as np
import mysql.connector
import sys

In [3]:
# подключение к базе opencorpora
db = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="root",
  database="opencopora.ru"
)
sql = db.cursor()

In [4]:
# составляем статистику по всему датасету
# количество статей
dataset_folders = os.listdir('dataset')
len(dataset_folders)

1964

In [ ]:
#for folder in dataset_folders:
    


/Users/frost/.conda/envs/opencopora-dataset/lib/python3.5/site-packages/ipykernel_launcher.py:10: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  # Remove the CWD from sys.path while we load stuff.
/Users/frost/.conda/envs/opencopora-dataset/lib/python3.5/site-packages/ipykernel_launcher.py:13: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  del sys.path[0]


{'auto_split_count': 42,
 'auto_split_misses_count': 8,
 'auto_split_score': 85.71,
 'doc_id': '2',
 'manual_split_count': 44,
 'manual_split_misses_count': 8,
 'manual_split_score': 81.82,
 'total_hit_count': 36}

In [31]:
# оценить качество разбиения текста на предложения (от 0 до 100 %)
def calculate_split_score(sentence_count, hit_count):
    if hit_count == 0:
        return 0
    return round((hit_count / sentence_count) * 100, 2) 

# подсчитать статистику точности разбиения на предложения по одному документу
def calculate_document_stats(doc_id):
    manual_sentences_file = 'dataset/' + doc_id + '/' + doc_id + '-sentences.csv'
    manual_df = pd.DataFrame.from_csv(path=manual_sentences_file)
    
    auto_sentences_file = 'dataset/' + doc_id + '/' + doc_id + '-sentences-auto.csv'
    auto_df = pd.DataFrame.from_csv(path=auto_sentences_file)
    
    # получаем списки предложений из обучающей и тестовой выборки
    manual_sentences = list(manual_df['source'].values)
    auto_sentences   = list(auto_df['source'].values)
    
    # всего совпадений (одинаково разбитых предложений)
    total_hits           = set(manual_sentences).intersection(set(auto_sentences))
    # определяем сколько (и какие) предложения из ручного разбиения удалось автоматически повторить
    manual_split_hits    = set(manual_sentences).intersection(set(auto_sentences))
    manual_split_misses  = set(manual_sentences).difference(set(manual_split_hits))
    manual_split_matches = [(sentence in manual_sentences) for sentence in manual_sentences]
    
    # определяем сколько (и какие) предложения из автоматического разбиения совпадают с ручным
    auto_split_hits    = set(auto_sentences).intersection(set(manual_sentences))
    auto_split_misses  = set(manual_sentences).difference(set(manual_split_hits))
    auto_split_matches = [(sentence in manual_sentences) for sentence in manual_sentences]
    
    return {
        "doc_id"                   : doc_id,
        # общее количество предложений которые одинаково разбиты вручную и автоматически
        "total_hit_count"          : len(total_hits),
        # результаты ручного разбиения
        "manual_split_count"       : len(manual_sentences),
        "manual_split_misses_count": len(manual_split_misses),
        # сколько процентов ручного разбиения удалось повторить автоматически
        "manual_split_score"       : calculate_split_score(len(manual_sentences), len(manual_split_hits)),
        # результаты авторазбиения
        "auto_split_count"         : len(auto_sentences),
        "auto_split_misses_count"  : len(auto_split_misses),
        # сколько процентов автоматического разбиения соответствует ручной проверке
        "auto_split_score"         : calculate_split_score(len(auto_sentences), len(auto_split_hits)),   
    }

/Users/frost/.conda/envs/opencopora-dataset/lib/python3.5/site-packages/ipykernel_launcher.py:10: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  # Remove the CWD from sys.path while we load stuff.
/Users/frost/.conda/envs/opencopora-dataset/lib/python3.5/site-packages/ipykernel_launcher.py:13: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  del sys.path[0]


{'auto_split_count': 42,
 'auto_split_misses_count': 8,
 'auto_split_score': 85.71,
 'doc_id': '2',
 'manual_split_count': 44,
 'manual_split_misses_count': 8,
 'manual_split_score': 81.82,
 'total_hit_count': 36}

In [33]:
calculate_document_stats('2')

/Users/frost/.conda/envs/opencopora-dataset/lib/python3.5/site-packages/ipykernel_launcher.py:10: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  # Remove the CWD from sys.path while we load stuff.
/Users/frost/.conda/envs/opencopora-dataset/lib/python3.5/site-packages/ipykernel_launcher.py:13: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  del sys.path[0]


{'auto_split_count': 42,
 'auto_split_misses_count': 8,
 'auto_split_score': 85.71,
 'doc_id': '2',
 'manual_split_count': 44,
 'manual_split_misses_count': 8,
 'manual_split_score': 81.82,
 'total_hit_count': 36}

In [19]:




for manual_index, manual_row in manual_df.iterrows():
    for auto_index, auto_row in auto_df.iterrows():
        # предложение из ручного разбиения найдено в автоматическом 
        if manual_row.source == auto_row.source:
            
            manual_splits_hit.append(manual_row)
            correct_auto_splits.append(auto_row)
            break

array(['13-я экспедиция МКС 30 марта 2006 года с космодрома Байконур в Казахстане стартует в космос экипаж 13-й долговременной экспедиции Международной космической станции.',
       'После более чем трёхлетнего перерыва долговременная экспедиция МКС будет вновь состоять из трёх космонавтов.',
       'Два первых члена экспедиции, командир Павел Виноградов (Россия) и борт-инженер Джеффри Уильямс, США отправляются на работу в космос 30 марта на корабле «Союз ТМА-8».',
       'Третий член экспедиции, астронавт Европейского космического агентства Томас Райтер (Германия) присоединится к первым двум в июле.',
       'Райтер прибудет на МКС на американском шаттле «Дискавери» (STS-121).',
       'Райтер — первый представитель Европы, который будет работать на МКС в составе долговременной экспедиции, до сегодняшнего дня в состав таких экспедиций входили только космонавты России и астронавты США.',
       'Павел Виноградов и Джеффри Уильямс возвратятся на Землю 24 сентября на своём «Союзе ТМА-8»,